In [ ]:
!pip install gdown
!gdown --fuzzy https://drive.google.com/file/d/1h9SvDOcJqEN7Ee-lBSoAOCuHxVxDHD8P/view?usp=sharing
!pip install catboost

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
final_df = pd.read_feather("bert-tiny-1500-final-df-lower.feather").dropna(subset=['price'])

In [3]:
embeddings = list(final_df.columns[:312])
X_train, X_test, y_train, y_test = train_test_split(final_df[embeddings], final_df["okpd2_value"],
                                                    test_size=0.05,
                                                    random_state=56,
                                                    shuffle=True)

In [4]:
len(X_test)

45307

In [5]:
from catboost import CatBoostClassifier

In [6]:
params = {'n_estimators' : 1000,
          'learning_rate': .01,
          'max_depth' : 6,
          'auto_class_weights': 'Balanced',
          'random_state': 42,
          "task_type": "GPU",
          "eval_metric": "Accuracy"
         }

model = CatBoostClassifier(**params)

In [7]:
model.fit(X_train, y_train, verbose=100, eval_set=(X_test, y_test), early_stopping_rounds=200)

0:	learn: 0.2410773	test: 0.2938188	best: 0.2938188 (0)	total: 605ms	remaining: 10m 4s
100:	learn: 0.7594150	test: 0.6250602	best: 0.6250602 (100)	total: 1m 1s	remaining: 9m 3s
200:	learn: 0.8022136	test: 0.6629203	best: 0.6629203 (200)	total: 2m 8s	remaining: 8m 29s
300:	learn: 0.8267182	test: 0.6745224	best: 0.6760311 (290)	total: 3m 15s	remaining: 7m 34s
400:	learn: 0.8430124	test: 0.6848976	best: 0.6850657 (399)	total: 4m 20s	remaining: 6m 29s
500:	learn: 0.8550263	test: 0.6848807	best: 0.6850881 (468)	total: 5m 22s	remaining: 5m 21s
600:	learn: 0.8632168	test: 0.6929083	best: 0.6929657 (587)	total: 6m 23s	remaining: 4m 14s
700:	learn: 0.8698004	test: 0.6942148	best: 0.6949054 (655)	total: 7m 23s	remaining: 3m 9s
800:	learn: 0.8755097	test: 0.6958442	best: 0.6958442 (800)	total: 8m 24s	remaining: 2m 5s
900:	learn: 0.8801029	test: 0.7032486	best: 0.7033073 (884)	total: 9m 22s	remaining: 1m 1s
999:	learn: 0.8843944	test: 0.6964000	best: 0.7036084 (918)	total: 10m 21s	remaining: 0us
b

In [8]:
model.save_model("okpd2-cat-model-1500-lower.cbm", format="cbm")